In [9]:
#IMPORT LIBRARIES

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import sent_tokenize
from textblob import TextBlob
nltk.download('gutenberg') #needed to access the raw text of a book
nltk.download('punkt') #needed to tokenize sentences
nltk.download('vader_lexicon') #NLTK's vader tool relies on a sentiment lexicon!

from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier


[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
# Load numeric data from Sheet1
num_df = pd.read_excel('Proj2Data.xlsx', sheet_name='Numeric Data')
num_df.head(10)

,day,open_price,high_price,low_price,moving_average_5_day,moving_average_10_day,moving_average_50_day,moving_average_200_day,volume,next_day_close_price
0,1,44.55,44.83,44.32,44.36,43.82,40.23,42.25,1500700,44.83
1,2,44.89,45.12,44.17,44.37,44.06,40.32,42.27,2391800,45.03
2,3,45.04,45.35,44.84,44.57,44.21,40.42,42.27,1723400,44.97
3,4,45.02,45.15,44.70,44.65,44.40,40.50,42.28,1490500,44.91
4,5,44.75,45.04,44.65,44.76,44.58,40.59,42.28,1349500,45.31
5,6,45.04,45.35,44.53,44.88,44.62,40.69,42.29,1707700,45.66
6,7,45.24,45.83,45.24,45.01,44.69,40.78,42.30,2389800,45.49
7,8,45.85,46.20,45.48,45.18,44.87,40.87,42.31,3256800,45.21
8,9,45.38,45.60,45.17,45.27,44.96,40.97,42.31,1955500,44.93
9,10,45.31,45.44,44.91,45.32,45.04,41.06,42.32,2211100,44.83


In [3]:
# Load text data (tweets) from Sheet2
text_df = pd.read_excel('Proj2Data.xlsx', sheet_name='Tweets')
text_df.head(10)

,day,tweet
0,1,"#Dan ($Dan) Doubles Down on Healthy, Eco-Frien..."
1,1,RT @DvdndDiplomats: Bert's X Always Buy stocks...
2,1,$Dan Alert From our Stock News Alerts App
3,1,X NEW Stocks at #FusionIQ with Master Scores >...
4,1,"#AmazonPrime creates a captive audience, so ""b..."
5,1,$Dan on watch if this pulls back I will be in....
6,1,Weekly S&P100 #Stocks Trend $Dan @
7,1,"How #Dan is upgrading its #healthy, #eco-frien..."
8,1,Why Big #Retailers Are Going Solar: $Dan
9,1,Dan Co. $Dan Given Buy Rating at Piper Jaffray


In [ ]:
# Tweet Data Cleaning


In [16]:
#USING TEXT BLOB
#define a few collections to hold polarity and subjectivity scores for each sentence
polarities = []
subjectivities = []

#get polarity and subjectivity scores for each sentence
for sentence in text_df.tweet:
  #the str() function is used to ensure that the input value is a string
  blob = TextBlob(str(sentence)) 
  #get polarity and subjectivity scores
  polarity, subjectivity = blob.polarity, blob.subjectivity
  #print('sentence: "{}", polarity: {:.3f}, subjectivity: {:.3f}'.format(sentence, polarity, subjectivity))
  polarities.append(polarity)
  subjectivities.append(subjectivity)

#calculate and display average polarity and subjectivity scores for all sentences
print('average polarity: {:.3f}'.format(np.mean(polarities)))
print('average subjectivity: {:.3f}'.format(np.mean(subjectivities)))


average polarity: 0.059
average subjectivity: 0.269


In [19]:
#USING NLTK
#define a sentiment analyzer object
analyzer = SentimentIntensityAnalyzer()

#apply sentiment analysis to each tweet in the dataframe and calculate average scores
text_df['sentiment_scores'] = text_df['tweet'].apply(lambda tweet: analyzer.polarity_scores(str(tweet)))


#extract individual sentiment scores
text_df['neg_score'] = text_df['sentiment_scores'].apply(lambda score: score['neg'])
text_df['neu_score'] = text_df['sentiment_scores'].apply(lambda score: score['neu'])
text_df['pos_score'] = text_df['sentiment_scores'].apply(lambda score: score['pos'])
text_df['compound_score'] = text_df['sentiment_scores'].apply(lambda score: score['compound'])

#calculate average scores
average_scores = text_df[['neg_score', 'neu_score', 'pos_score', 'compound_score']].mean()

#print average sentiment scores
print('Average sentiment scores - negative: {:.3f}, neutral: {:.3f}, positive: {:.3f}, overall: {:.3f}'.format(
    average_scores['neg_score'], average_scores['neu_score'], average_scores['pos_score'], average_scores['compound_score']))


Average sentiment scores - negative: 0.036, neutral: 0.872, positive: 0.092, overall: 0.125
